# Section 5

In [1]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae

class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

Implement the mean absolute error:
$$
MAE = \frac{1}{N}\sum_{i=1}^N |y_i-x_i^\top\theta|
$$

In [15]:
def get_MAE(theta, X, y):
    mae = np.average(np.abs(y - X@theta.T), axis=0)
    return mae

## Question 7.2
Implement the problem from Question 7.1 (Use the `GLPK` solver)

In [22]:
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()
X, X_test, Y, Y_test = train_test_split(diabetes['data'], 
                                        np.expand_dims(diabetes['target'], 1), 
                                        test_size=0.25, random_state=0)
X = np.concatenate((X, np.ones((X.shape[0], 1))), axis=1) # Bias column:
X_test = np.concatenate((X_test, np.ones((X_test.shape[0], 1))), axis=1)

train, validation, thetas, optimal = [], [], [], []

In [23]:
# Hyperparameter:
lambda_ = np.logspace(-5, -1, 50, base = 10)

#LP solver:
def LP_solver(X, Y, lambda_, k):
    d = X.shape[1]
    N = X.shape[0]
    Beta = cp.Variable((N, 1))
    b = cp.Variable((d, 1))
    alpha = cp.Variable((1,1))
    theta = cp.Variable((1, d))
    LP = cp.Problem( cp.Minimize(k * alpha + cp.sum(Beta) + lambda_ * cp.sum(b)),               [
        alpha + Beta >= 1/k * (Y - X @ theta.T),
        alpha + Beta >= -1/k * (Y - X @ theta.T),
        Beta >= 0,
        -b <= theta,
        theta <= b
    ])
    LP.solve()
    optimal_theta = theta.value
    optimal_value = LP.value
    dual_value = LP.constraints[0].dual_value
    return optimal_theta,  optimal_value, dual_value


In [27]:
# Cross-validation:
for l in lambda_:
    optimal_theta, optimal_value, dual_value = LP_solver(X, Y, l, math.floor(0.75 * X.shape[0]))
    thetas.append(optimal_theta)
    optimal.append(optimal_value)
    train.append(get_MAE(optimal_theta, X, Y))
    validation.append(get_MAE(optimal_theta, X_test, Y_test))
    
best_lambda = lambda_[np.argmin(validation)]
best_theta = thetas[np.argmin(validation)]
best_value = optimal[np.argmin(validation)]

print('---Optimal values--')
print(f'Optimal lambda: {best_lambda}')
print(f'Optimal value: {best_value}')
print(f'Optimal theta:\n {best_theta}')

---Optimal values--
Optimal lambda: 0.004094915062380423
Optimal value: 69.85032419743328
Optimal theta:
 [[ -51.25036457 -256.59297979  256.59298077  256.59298071  134.15685779
  -256.59297983 -256.59298057  256.59298031  256.59298072  249.77586678
   151.12806908]]


In [28]:
print('MAE of training : {}'.format(get_MAE(best_theta, X, Y)))

MAE of training : [46.42683702]


In [29]:
print('MAE of test : {}'.format(get_MAE(best_theta, X_test, Y_test)))

MAE of test : [43.17935494]


There is a significant difference between the MEA of the training set and the MEA of the test set.